In [1]:
# imports
import math
import os
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch.nn.functional as F
from dataset_utils import alternativeSeperation

This part seperates the validation folder and enables utilization of PyTorch's ImageDataset implementation. Please note that this part 

In [2]:
#alternativeSeperation()

In [3]:
from model_utils import ModelEngine
transformations = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        # transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}

data_dir = "data"

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transformations[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32,
                             shuffle=True)
               for x in ['train', 'val']}



C:\Users\Furkan\PycharmProjects\NetworkEvolver\model_utils.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
class SGDR(_LRScheduler):

    def __init__(self, optimizer, t_max, eta_min=1e-6, last_epoch=-1, t_mult=2, decay_mult = .8):
        self.decay_mult = decay_mult
        self.t_max = t_max
        self.t_mult = t_mult
        self.restart_point = t_max
        self.eta_min = eta_min
        self.restarted_at = 0
        super().__init__(optimizer, last_epoch)
        
    def restart(self):
        self.restart_point *= self.t_mult
        self.restarted_at = self.last_epoch
        self.t_max *= self.decay_mult

    def cosine(self, base_lr):
        return self.eta_min + (base_lr - self.eta_min) * (1 + math.cos(math.pi * (self.last_epoch - self.restarted_at) / self.restart_point)) / 2

    def get_lr(self):
        if (self.last_epoch - self.restarted_at) >= self.restart_point:
            self.restart()
        return [self.cosine(base_lr) for base_lr in self.base_lrs]

In [5]:
class ConvNet1(nn.Module):
    def __init__(self):
        super(ConvNet1, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),    
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),  
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2),  
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
        )
        self.fc = nn.Linear(512 * 11 * 11, 200)
        
    def forward(self, inputs):
        x = self.feature_extractor(inputs)
        return self.fc(x.view(x.size(0), -1))

In [6]:
model1 = ConvNet1()
criterion1 = nn.CrossEntropyLoss()
optimizer1 = optim.SGD(model1.parameters(), lr=.01, momentum=.9)
scheduler1 = SGDR(optimizer1, len(dataloaders["train"]))
engine1 = ModelEngine(model1, criterion1, optimizer1, scheduler=scheduler1)

Using device cuda:0


In [ ]:
engine1.fit(dataloaders)

In [5]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, batch_norm=True):
        super(ResBlock, self).__init__()

        padding_size = kernel_size // 2

        self.conv1 = nn.Conv2d(in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=kernel_size,
                               padding=padding_size,
                               stride=stride)

        self.conv2 = nn.Conv2d(in_channels=out_channels,
                               out_channels=out_channels,
                               kernel_size=kernel_size,
                               padding=padding_size,
                               stride=stride)

        self.batch_norm = batch_norm
        if batch_norm:
            self.batch_norm_layer = nn.BatchNorm2d(out_channels)
        self.size_fix = None
        if in_channels != out_channels:
            self.size_fix = nn.Sequential(
                nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels))
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, inputs):
        residual = inputs
        x = self.conv1(inputs)
        x = self.batch_norm_layer(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batch_norm_layer(x)
        if self.size_fix:
            residual = self.size_fix(residual)
        return self.relu(x + residual)

class ResNet1(nn.Module):
    def __init__(self):
        super(ResNet1, self).__init__()
        self.feature_extractor = nn.Sequential(
            ResBlock(in_channels=3, out_channels=128, kernel_size=3, stride=1),
#             nn.MaxPool2d(2, 2),
            ResBlock(in_channels=128, out_channels=256, kernel_size=3, stride=1),
            nn.MaxPool2d(2, 2),
            ResBlock(in_channels=256, out_channels=512, kernel_size=3, stride=1),
        )
        self.fc = nn.Linear(512 * 32 * 32, 200)
        
    def forward(self, inputs):
        x = self.feature_extractor(inputs)
        return self.fc(x.view(x.size(0), -1))

In [6]:
model2 = ResNet1()
criterion2 = nn.CrossEntropyLoss()
optimizer2 = optim.AdamW(model2.parameters())#optim.SGD(model2.parameters(), lr=.01, momentum=.9)
scheduler2 = SGDR(optimizer2, len(dataloaders["train"]))
engine2 = ModelEngine(model2, criterion2, optimizer2)#, scheduler=scheduler2)

Using device cuda:0


In [7]:
engine2.fit(dataloaders)

NameError: name 'earlyStopping' is not defined